# load package and data

In [1]:
import json
from collections import Counter

import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

In [2]:
def load_data():
    with open('song.json','rt') as inf:
        data = json.load(inf)
    data =pd.DataFrame(data)
    data.set_index('id',inplace=True)
    data['time_played']=pd.to_datetime(data.time_played)
    data['user_sign_up_date']=pd.to_datetime(data.user_sign_up_date)
    
    return data

In [3]:
data = load_data()

In [5]:
data.shape

(4000, 5)

In [85]:
data.head()

,song_played,time_played,user_id,user_sign_up_date,user_state
id,,,,,
GOQMMKSQQH,Hey Jude,2015-06-11 21:51:35,122,2015-05-16,Louisiana
HWKKBQKNWI,We Can Work It Out,2015-06-06 16:49:19,3,2015-05-01,Ohio
DKQSXVNJDH,Back In the U.S.S.R.,2015-06-14 02:11:29,35,2015-05-04,New Jersey
HLHRIDQTUW,P.s. I Love You,2015-06-08 12:26:10,126,2015-05-16,Illinois
SUKJCSBCYW,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00,6,2015-05-01,New Jersey


# Q1

In [19]:
user_counts = data.groupby('user_state').user_id.agg(lambda ids:len(np.unique(ids)))

In [20]:
user_counts.sort_values(ascending=False,inplace=True)

In [21]:
user_counts[:3]

user_state
New York      23
California    21
Texas         15
Name: user_id, dtype: int64

In [22]:
user_counts[-3:]

user_state
Connecticut    1
New Mexico     1
Arizona        1
Name: user_id, dtype: int64

# Q2
I define 'average play event per hour' as a metric to measure user engagement of a state.

In [50]:
data.head()

,song_played,time_played,user_id,user_sign_up_date,user_state
id,,,,,
GOQMMKSQQH,Hey Jude,2015-06-11 21:51:35,122,2015-05-16,Louisiana
HWKKBQKNWI,We Can Work It Out,2015-06-06 16:49:19,3,2015-05-01,Ohio
DKQSXVNJDH,Back In the U.S.S.R.,2015-06-14 02:11:29,35,2015-05-04,New Jersey
HLHRIDQTUW,P.s. I Love You,2015-06-08 12:26:10,126,2015-05-16,Illinois
SUKJCSBCYW,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00,6,2015-05-01,New Jersey


In [62]:
def count_by_state(df):
    total_played = df.shape[0]
    first_play_dt = df.time_played.min()
    last_play_dt = df.time_played.max()
    duration = last_play_dt - first_play_dt
    duration_hours = duration.total_seconds()/3600
    return pd.Series([first_play_dt,last_play_dt,duration,duration_hours,total_played],
                    index=['first_play_dt','last_play_dt','duration','duration_hours','total_played'])

In [63]:
counts_by_states = data.groupby('user_state').apply(count_by_state)

In [64]:
counts_by_states.head()

,first_play_dt,last_play_dt,duration,duration_hours,total_played
user_state,,,,,
Alabama,2015-06-01 14:17:56,2015-06-28 14:48:55,27 days 00:30:59,648.516389,104
Alaska,2015-06-01 18:48:18,2015-06-28 22:58:23,27 days 04:10:05,652.168056,58
Arizona,2015-06-01 13:05:17,2015-06-28 19:23:45,27 days 06:18:28,654.307778,22
Arkansas,2015-06-01 14:07:37,2015-06-28 17:40:11,27 days 03:32:34,651.542778,34
California,2015-06-01 06:33:03,2015-06-28 20:35:50,27 days 14:02:47,662.046389,425


In [65]:
counts_by_states['hr_average']= counts_by_states.total_played/counts_by_states.duration_hours

In [67]:
counts_by_states.sort_values(by='hr_average',ascending=False,inplace=True)

In [69]:
counts_by_states[:3]

,first_play_dt,last_play_dt,duration,duration_hours,total_played,hr_average
user_state,,,,,,
New York,2015-06-01 06:14:45,2015-06-28 21:36:40,27 days 15:21:55,663.365278,469,0.707001
California,2015-06-01 06:33:03,2015-06-28 20:35:50,27 days 14:02:47,662.046389,425,0.641949
Texas,2015-06-01 06:09:04,2015-06-28 20:28:35,27 days 14:19:31,662.325278,230,0.347261


In [70]:
counts_by_states[-3:]

,first_play_dt,last_play_dt,duration,duration_hours,total_played,hr_average
user_state,,,,,,
Connecticut,2015-06-06 19:23:58,2015-06-28 13:16:32,21 days 17:52:34,521.876111,16,0.030659
New Mexico,2015-06-01 05:22:30,2015-06-28 13:15:58,27 days 07:53:28,655.891111,17,0.025919
Kansas,2015-06-05 15:01:50,2015-06-27 09:02:15,21 days 18:00:25,522.006944,8,0.015325


# Q3

## solution 1

In [116]:
user_sign = data.groupby('user_state')[['user_id','user_sign_up_date']].agg({'user_sign_up_date':'min'}).reset_index()

In [117]:
user_sign.head()

user_state user_sign_up_date                  
                        user_id user_sign_up_date
0     Alabama                 5        2015-05-01
1      Alaska               106        2015-05-12
2     Arizona               105        2015-05-12
3    Arkansas                78        2015-05-08
4  California                39        2015-05-04

In [120]:
pd.DataFrame(user_sign.values,columns=['user_state','user_id','sign_date']).sort_values(by='sign_date').head()

,user_state,user_id,sign_date
0,Alabama,5,2015-05-01 00:00:00
35,Texas,7,2015-05-01 00:00:00
30,Oregon,1,2015-05-01 00:00:00
28,Ohio,3,2015-05-01 00:00:00
26,North Carolina,2,2015-05-01 00:00:00


## solution 2

In [121]:
def first_sign(df):
    idx=df.user_sign_up_date.argmin()
    return df.loc[idx,["user_id","user_sign_up_date"]]

In [123]:
out = data.groupby('user_state').apply(first_sign)

C:\Users\54308\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:2: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  


In [124]:
out.sort_values(by='user_sign_up_date')

,user_id,user_sign_up_date
user_state,,
Alabama,5,2015-05-01
Texas,7,2015-05-01
Oregon,1,2015-05-01
Ohio,3,2015-05-01
North Carolina,2,2015-05-01
New Mexico,4,2015-05-01
New Jersey,6,2015-05-01
Pennsylvania,11,2015-05-02
New York,19,2015-05-02


# Q4

## matrix solution 1

In [129]:
users=data.user_id.unique().tolist()

In [139]:
song_m = np.zeros((len(np.unique(data.song_played)),len(users)),dtype=np.int)

In [140]:
song_m_df = pd.DataFrame(song_m,index=np.unique(data.song_played),columns=sorted(users)) 

In [141]:
song_m_df.head()

,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
A Day In The Life,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A Hard Day's Night,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A Saturday Club Xmas/Crimble Medley,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ANYTIME AT ALL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Across The Universe,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [159]:
a,b = data[['song_played','user_id']].iloc[0,:]

In [162]:
for i in range(len(data)):
    s,u = data[['song_played','user_id']].iloc[i,:]
    song_m_df.loc[s,u]+=1

In [163]:
song_m_df.head()

,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
A Day In The Life,0,0,1,3,0,2,0,0,0,0,...,0,0,3,3,0,2,0,0,2,0
A Hard Day's Night,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
A Saturday Club Xmas/Crimble Medley,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ANYTIME AT ALL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Across The Universe,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## matrix solution 2

In [170]:
def count_user(df):
    return pd.Series(Counter(df.user_id))

In [186]:
song_matrix= data.groupby('song_played').apply(count_user).unstack(fill_value=0)

In [193]:
song_matrix.head()

,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,0,0,1,3,0,2,0,0,0,0,...,0,0,3,3,0,2,0,0,2,0
A Hard Day's Night,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
A Saturday Club Xmas/Crimble Medley,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ANYTIME AT ALL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Across The Universe,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## cosine similarity
### cosine similarity is XY/|X||Y|

In [189]:
# normalize the matrix, then denominator become 1
from sklearn.preprocessing import normalize
song_m_norm = normalize(song_matrix,axis=1)

In [192]:
# dot 
song_cos = song_m_norm.dot(song_m_norm.T)

In [197]:
song_cos.shape,song_matrix.shape

((100, 100), (100, 196))

In [198]:
song_rec_m = pd.DataFrame(song_cos,index=song_matrix.index,columns=song_matrix.index)

In [199]:
song_rec_m

song_played,A Day In The Life,A Hard Day's Night,A Saturday Club Xmas/Crimble Medley,ANYTIME AT ALL,Across The Universe,All My Loving,All You Need Is Love,And Your Bird Can Sing,BAD BOY,BALLAD OF JOHN AND YOKO,...,We Can Work It Out,When I'm 64,While My Guitar Gently Weeps,Wild Honey Pie,With a Little Help From My Friends,YOUR MOTHER SHOULD KNOW,Yellow Submarine,Yesterday,You Never Give Me Your Money,You're Going To Lose That Girl
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,1.000000,0.235702,0.074536,0.119523,0.212132,0.355023,0.329404,0.152145,0.210819,0.172133,...,0.464938,0.030429,0.508964,0.223607,0.359092,0.037268,0.318198,0.353220,0.087841,0.000000
A Hard Day's Night,0.235702,1.000000,0.000000,0.000000,0.100000,0.136931,0.111803,0.000000,0.000000,0.091287,...,0.259548,0.129099,0.210099,0.000000,0.000000,0.000000,0.050000,0.195468,0.074536,0.000000
A Saturday Club Xmas/Crimble Medley,0.074536,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.109435,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ANYTIME AT ALL,0.119523,0.000000,0.000000,1.000000,0.000000,0.154303,0.094491,0.109109,0.000000,0.000000,...,0.116991,0.000000,0.138107,0.089087,0.183942,0.000000,0.000000,0.146845,0.000000,0.000000
Across The Universe,0.212132,0.100000,0.000000,0.000000,1.000000,0.091287,0.000000,0.000000,0.000000,0.000000,...,0.138426,0.000000,0.116722,0.000000,0.000000,0.000000,0.000000,0.043437,0.000000,0.000000
All My Loving,0.355023,0.136931,0.000000,0.154303,0.091287,1.000000,0.204124,0.353553,0.136083,0.083333,...,0.205343,0.000000,0.309001,0.048113,0.198680,0.144338,0.273861,0.178437,0.068041,0.000000
All You Need Is Love,0.329404,0.111803,0.000000,0.094491,0.000000,0.204124,1.000000,0.000000,0.166667,0.204124,...,0.174110,0.000000,0.313197,0.176777,0.162221,0.088388,0.111803,0.291386,0.166667,0.000000
And Your Bird Can Sing,0.152145,0.000000,0.000000,0.109109,0.000000,0.353553,0.000000,1.000000,0.192450,0.000000,...,0.178707,0.000000,0.195893,0.136083,0.140488,0.102062,0.258199,0.112154,0.000000,0.000000
BAD BOY,0.210819,0.000000,0.000000,0.000000,0.000000,0.136083,0.166667,0.192450,1.000000,0.136083,...,0.103176,0.192450,0.191398,0.157135,0.108148,0.000000,0.000000,0.064752,0.111111,0.000000


## recommendations

In [ ]:
# solution 1

In [215]:
def rec(song,df):
    print(df[song].sort_values(ascending=False).index[1])

In [216]:
rec('A Day In The Life',song_rec_m)

Come Together


In [220]:
# solution 2
def rec(df,song,k):
    rec_df = df[song].sort_values(ascending=False)[1:k+1].reset_index()
    return rec_df.rename(columns={'song_played': 'Song', song: 'Similarity'})

In [222]:
rec(song_rec_m,'A Day In The Life',10)

,Song,Similarity
0,Come Together,0.577357
1,Revolution,0.575179
2,Get Back,0.532342
3,While My Guitar Gently Weeps,0.508964
4,Back In the U.S.S.R.,0.492563
5,Hey Jude,0.490362
6,Let It Be,0.481286
7,Here Comes The Sun,0.475850
8,Lucy In The Sky With Diamonds,0.470427
9,Hello Goodbye,0.466242


# backup

In [ ]:
# knowledge point
1. lambda(x:len(np.unique(x)))
2. df.diff(1) = df.shift(1)-df
3. df.time.seconds 只计算h:m:s 的部分
4. df.time.total_seconds() 同时计算 d 和 h:m:s
5. sorted(lt,key=lambda lt:lt[2])
6. Counter(df.user_id) #生成的是字典
7. pd.Series(Counter(df.user_id)) #把字典变成了序列
8. pd.Series(Counter(df.user_id)).unstack(fill_value=0) #将序列pivot了，然后空白处用0填上